In [ ]:
!pip install flask-ngrok

In [ ]:
from flask import Flask
from flask_ngrok import run_with_ngrok
from flask import render_template, url_for, request, jsonify      
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
import pickle
import numpy as np
import pandas as pd
from csv import writer

app = Flask(__name__,template_folder='/content/drive/MyDrive/MINOR2/Flask app/templates')
run_with_ngrok(app)


with open(r"/content/drive/MyDrive/MINOR2/Flask app/toxic_vect.pkl", "rb") as f:
    tox = pickle.load(f)

with open(r"/content/drive/MyDrive/MINOR2/Flask app/severe_toxic_vect.pkl", "rb") as f:
    sev = pickle.load(f)

with open(r"/content/drive/MyDrive/MINOR2/Flask app/obscene_vect.pkl", "rb") as f:
    obs = pickle.load(f)

with open(r"/content/drive/MyDrive/MINOR2/Flask app/insult_vect.pkl", "rb") as f:
    ins = pickle.load(f)

with open(r"/content/drive/MyDrive/MINOR2/Flask app/threat_vect.pkl", "rb") as f:
    thr = pickle.load(f)

with open(r"/content/drive/MyDrive/MINOR2/Flask app/identity_hate_vect.pkl", "rb") as f:
    ide = pickle.load(f)

with open(r"/content/drive/MyDrive/MINOR2/Flask app/toxic_model.pkl", "rb") as f:
    tox_model = pickle.load(f)

with open(r"/content/drive/MyDrive/MINOR2/Flask app/severe_toxic_model.pkl", "rb") as f:
    sev_model = pickle.load(f)

with open(r"/content/drive/MyDrive/MINOR2/Flask app/obscene_model.pkl", "rb") as f:
    obs_model  = pickle.load(f)

with open(r"/content/drive/MyDrive/MINOR2/Flask app/insult_model.pkl", "rb") as f:
    ins_model  = pickle.load(f)

with open(r"/content/drive/MyDrive/MINOR2/Flask app/threat_model.pkl", "rb") as f:
    thr_model  = pickle.load(f)

with open(r"/content/drive/MyDrive/MINOR2/Flask app/identity_hate_model.pkl", "rb") as f:
    ide_model  = pickle.load(f)

@app.route("/")
def home():
    return render_template('index_toxic.html')

@app.route("/predict", methods=['POST'])
def predict():
    
    # Take a string input from user
    user_input = request.form['text']
    data = [user_input]

    vect = tox.transform(data)
    pred_tox_p = tox_model.predict_proba(vect)[:,1]
    pred_tox = tox_model.predict(vect)[0]


    vect = sev.transform(data)
    pred_sev_p = sev_model.predict_proba(vect)[:,1]
    pred_sev = sev_model.predict(vect)[0]


    vect = obs.transform(data)
    pred_obs_p = obs_model.predict_proba(vect)[:,1]
    pred_obs = obs_model.predict(vect)[0]

    vect = thr.transform(data)
    pred_thr_p = thr_model.predict_proba(vect)[:,1]
    pred_thr = thr_model.predict(vect)[0]

    vect = ins.transform(data)
    pred_ins_p = ins_model.predict_proba(vect)[:,1]
    pred_ins = ins_model.predict(vect)[0]

    vect = ide.transform(data)
    pred_ide_p = ide_model.predict_proba(vect)[:,1]
    pred_ide = ide_model.predict(vect)[0]

    out_tox = round(pred_tox_p[0], 2)
    out_sev = round(pred_sev_p[0], 2)
    out_obs = round(pred_obs_p[0], 2)
    out_ins = round(pred_ins_p[0], 2)
    out_thr = round(pred_thr_p[0], 2)
    out_ide = round(pred_ide_p[0], 2)
    _tox= "Yes" if pred_tox==1 else "No"
    _sev= "Yes" if pred_sev==1 else "No"
    _obs= "Yes" if pred_obs==1 else "No"
    _ins= "Yes" if pred_ins==1 else "No"
    _thr= "Yes" if pred_thr==1 else "No"
    _ide= "Yes" if pred_ide==1 else "No"

    print(out_tox)
    print(pred_tox)

    return render_template('index_toxic.html', 
                            pred_tox = 'Prob (Toxic): {}'.format(out_tox),
                            pred_sev = 'Prob (Severe Toxic): {}'.format(out_sev), 
                            pred_obs = 'Prob (Obscene): {}'.format(out_obs),
                            pred_ins = 'Prob (Insult): {}'.format(out_ins),
                            pred_thr = 'Prob (Threat): {}'.format(out_thr),
                            pred_ide = 'Prob (Identity Hate): {}'.format(out_ide),
                            _tox='Toxic: {}'.format(_tox),
                           _sev='Severe Toxic: {}'.format(_sev),
                           _obs='Obscene: {}'.format(_obs),
                           _ins='Insult: {}'.format(_ins),
                           _thr='Threat: {}'.format(_thr),
                           _ide='Identity Hate: {}'.format(_ide)
                                                   
                            )
     
@app.route("/addtodatabase", methods=['POST'])
def addtodatabase():
   return render_template('addtodatabase.html')

@app.route("/addtodatabase/addinfo", methods=['GET','POST'])
def addinfo():
  
  comment=request.form['text']
  tox=request.form['toxic']
  severe=request.form['severe']
  obs=request.form['obs']
  ins=request.form['ins']
  thr=request.form['thr']
  ide=request.form['ide']
  df=pd.read_csv("/content/drive/MyDrive/MINOR2/Flask app/toxicdataextra.csv")
  id=int(df.iloc[-1][0])+1
  data=[id,comment,tox,severe,obs,ins,thr,ide]
  with open('/content/drive/MyDrive/MINOR2/Flask app/toxicdataextra.csv','a') as f_obj:
    writer_obj=writer(f_obj)
    writer_obj.writerow(data)
    f_obj.close()
  #df=df.append({"id":id,"comment_text":comment,"toxic":tox,"severe_toxic":severe,"obscene":obs,"insult":ins,"threat":thr,"identity_hate":ide},ignore_index=True)
  df.loc[len(df.index)]=[id,comment,tox,severe,obs,ins,thr,ide]
 
  df.to_csv("toxicdataextra.csv",index=False)
  print(df)
  return render_template('addtodatabase.html',res='Thanks for your Contribution.Data Added Successfully!')   

@app.route("/")
def goback():
   return render_template('index_toxic.html')
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://ed0bd0fdec3f.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [04/May/2021 05:19:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2021 05:19:35] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [04/May/2021 05:19:54] "POST /predict HTTP/1.1" 200 -


1.0
1


127.0.0.1 - - [04/May/2021 05:20:13] "POST /addtodatabase HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2021 05:20:53] "POST /addtodatabase/addinfo HTTP/1.1" 200 -


   id                     comment_text toxic  ... threat insult identity_hate
0    1             you are so dead now     1  ...      1      0             0
1    2                        fuck you     1  ...      1      1             0
2    3               you are so pretty     0  ...      0      0             0
3    4               You piece of shit     1  ...      1      0             1
4    5  why don't you go kill yourself     1  ...      0      1             1

[5 rows x 8 columns]


127.0.0.1 - - [04/May/2021 05:21:02] "GET / HTTP/1.1" 200 -
